In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

In [ ]:
feeding = pd.read_csv('dape_pen3_4.csv')

In [ ]:
feeding.head()

In [ ]:
feeding.tail()

In [ ]:
def get_adj(temp):
    if temp <= 4:
        return 0.6
    if temp <= 9:
        return 0.8
    if temp <= 17:
        return 1
    return 0.8

def get_adj2(temp):
    if temp <= 4:
        return 0.1
    if temp <= 6:
        return 0.6
    if temp <= 8:
        return 0.8
    if temp <= 10:
        return 1
    if temp <= 12:
        return 1.1
    if temp <= 14:
        return 1.2
    if temp <= 16:
        return 1.2
    if temp <= 18:
        return 0.8
    if temp <= 20:
        return 0.2
    return 0.0


In [ ]:
currWeight1 = 3983.5 - 200 
currWeight2 = 3983.5 - 200
currWeight3 = 3983.5 - 200

feedingWeight1 = []
feedingWeight2 = []
feedingWeight3 = []

for index, row in feeding.iterrows():
    feed = row['Feed Amount']
    FCR = row['FCR']
    numFish = row['Number of fish']
    temp = row['Temp']
    
    FCR1 = 1.2
    FCR2 = 1.3
    FCR3 = 1.4

    adj1 = get_adj(temp)
    adj2 = get_adj2(temp)
    
    feedingWeight1.append(currWeight1)
    feedingWeight2.append(currWeight2)
    feedingWeight3.append(currWeight3)
    
    if feed <= 0:
        continue
        
#     amt1 = feed / FCR2 / numFish * 1000
#     amt2 = feed / FCR2 / numFish * 1000 * adj1
#     amt3 = feed / FCR2 / numFish * 1000 * adj2
    
    amt1 = feed / FCR / numFish * 1000 * 1.2
    amt2 = feed / FCR / numFish * 1000 * adj1 * 1.2
    amt3 = feed / FCR / numFish * 1000 * adj2 * 1.2
    
    currWeight1 = currWeight1 + amt1
    currWeight2 = currWeight2 + amt2
    currWeight3 = currWeight3 + amt3
    
feeding['Calculated weight1'] = feedingWeight1
feeding['Calculated weight2'] = feedingWeight2
feeding['Calculated weight3'] = feedingWeight3

In [ ]:
from matplotlib.dates import DateFormatter, WeekdayLocator, MONDAY

fig, ax = plt.subplots(figsize=(20, 10))

ax.xaxis.set_major_locator(WeekdayLocator(MONDAY))
ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))

ax.set_xlabel('Date')
ax.plot(plot_dates, weights1, label = 'Old Model')
ax.plot(plot_dates, weights2, label = 'New Model')
ax.plot(plot_dates, weights3, label = 'Current Model')
ax.plot(plot_dates, feeding['Average weight'], label = 'Feeding Model')
ax.plot(plot_dates, feeding['Calculated weight1'], label = 'Calculated Model1')
ax.plot(plot_dates, feeding['Calculated weight2'], label = 'Calculated Model2')
ax.plot(plot_dates, feeding['Calculated weight3'], label = 'Calculated Model3')

ax.legend()

In [ ]:
plt.plot(feeding['Average weight'])
plt.plot(feedingWeight)

In [ ]:
plt.plot(feeding['Date'], feeding['Feed Amount'])

In [ ]:

    
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch
from research.weight_estimation.keypoint_utils import body_parts


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in body_parts.core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = body_parts.core_body_parts.index(body_parts.UPPER_LIP)
    tail_notch_idx = body_parts.core_body_parts.index(body_parts.TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input


"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

In [ ]:
from filter_optimization.filter_optimization_task import extract_biomass_data

df = extract_biomass_data(145, '2020-09-01', '2021-02-21', 0.01)

In [ ]:
import json, os
from research.utils.data_access_utils import S3AccessUtils

model1_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb'
model2_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb'

s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

weight_model1_f, _, _ = s3.download_from_url(model1_url)
weight_model2_f, _, _ = s3.download_from_url(model2_url)
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

weight_estimator1 = WeightEstimator(weight_model1_f, kf_model_f)
weight_estimator2 = WeightEstimator(weight_model2_f, kf_model_f)



In [ ]:
weights1 = []
weights2 = []

count = 0

for idx, row in df.iterrows():
    count = count + 1
    
    if count % 10000 == 0:
        print(count)
    
    annotation = row.annotation
    if not annotation:
        weights.append(None)
        continue
    camera_metadata = row.camera_metadata

    camera_metadata_obj = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )

    weight1, length, kf = weight_estimator1.predict(annotation, camera_metadata_obj)
    weight2, length, kf = weight_estimator2.predict(annotation, camera_metadata_obj)

    weights1.append(weight1)
    weights2.append(weight2)

In [ ]:
df['weights1'] = weights1
df['weights2'] = weights2

In [ ]:
dates = sorted(list(set(list(feeding['Date']))))
plot_dates = [ datetime.strptime(d, '%Y-%m-%d') for d in dates ]

weights1 = []
weights2 = []
weights3 = []

for date in dates:
    mask1 = (df.akpd_score > 0.95) & (df.hour >= 8) & (df.hour <= 16) & (df.date == date)
    mask2 = (df.akpd_score > 0.95) & (df.hour >= 8) & (df.hour <= 16) & (df.date == date)
    mask3 = (df.akpd_score > 0.95) & (df.hour >= 8) & (df.hour <= 16) & (df.date == date)
    
    weights1.append(np.mean(df[mask1]['weights1']))
    weights2.append(np.mean(df[mask2]['weights2']))
    weights3.append(np.mean(df[mask3]['estimated_weight_g']))
    


In [ ]:
from matplotlib.dates import DateFormatter, WeekdayLocator, MONDAY

fig, ax = plt.subplots(figsize=(20, 10))

ax.xaxis.set_major_locator(WeekdayLocator(MONDAY))
ax.xaxis.set_major_formatter(DateFormatter('%m/%d'))

ax.set_xlabel('Date')
ax.plot(plot_dates, weights1, label = 'Old Model')
ax.plot(plot_dates, weights2, label = 'New Model')
ax.plot(plot_dates, weights3, label = 'Current Model')
ax.plot(plot_dates, feeding['Average weight'], label = 'Feeding Model')
ax.plot(plot_dates, feeding['Calculated weight1'], label = 'Calculated Model1')
ax.plot(plot_dates, feeding['Calculated weight2'], label = 'Calculated Model2')
ax.plot(plot_dates, feeding['Calculated weight3'], label = 'Calculated Model3')

ax.legend()